[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/Pinkwjp/sentiment-analysis-with-transformer-on-IMDB)

In [ ]:
# TODO: fix open in colab

In [ ]:
# NOTE: python 3.12 not working too well with Keras
# pipenv install --python 3.10
# pipenv shell

In [ ]:
%pip install -q --upgrade keras-nlp  # install keras-nlp before keras
%pip install keras-tuner
%pip install -q --upgrade keras

In [2]:
import keras
from keras import layers
import keras_nlp
from keras_nlp import layers as nlp_layers
import keras_tuner

In [3]:
from pathlib import Path

trained_model_folder = Path("./trained_models/")

if not trained_model_folder.exists():
    trained_model_folder.mkdir()

assert trained_model_folder.exists()

In [5]:
vocab_size = 20000
maxlen = 200

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)
print("training set size:", len(x_train))
print("test set size:", len(x_test))
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.utils.pad_sequences(x_test, maxlen=maxlen)

training set size: 25000
test set size: 25000


In [6]:
inputs = keras.Input(shape=(None,), dtype="int64")

x = nlp_layers.TokenAndPositionEmbedding(vocabulary_size=vocab_size,
                                         sequence_length=maxlen,
                                         embedding_dim=32
                                         )(inputs)

x = nlp_layers.TransformerEncoder(intermediate_dim=20,
                                  num_heads=2
                                  )(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

baseline_model = keras.Model(inputs, outputs)
baseline_model.compile(optimizer="rmsprop",
                       loss="binary_crossentropy",
                       metrics=["accuracy"])
baseline_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 32)            │         646,400 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 32)            │           5,684 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 32)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 652,117 (2.49 MB)

 Trainable params: 652,117 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from keras import callbacks

def save_best_only(file_path: Path) -> list[callbacks.Callback]:
    return [callbacks.ModelCheckpoint(file_path, save_best_only=True)]

In [10]:
baseline_model_path = trained_model_folder / "baseline_model.keras"

baseline_model.fit(x_train, y_train,
                   batch_size=64, epochs=10,
                   validation_split=0.2,
                   callbacks=save_best_only(baseline_model_path))


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step - accuracy: 0.6162 - loss: 0.7256 - val_accuracy: 0.8232 - val_loss: 0.4089
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8273 - loss: 0.3946 - val_accuracy: 0.8482 - val_loss: 0.3430
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8579 - loss: 0.3321 - val_accuracy: 0.8484 - val_loss: 0.3463
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8755 - loss: 0.2949 - val_accuracy: 0.8454 - val_loss: 0.3642
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8929 - loss: 0.2603 - val_accuracy: 0.8426 - val_loss: 0.3799
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9075 - loss: 0.2263 - val_accuracy: 0.8364 - val_loss: 0.3990
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9177 - loss: 0.2078 - val_accuracy: 0.8482 - val_loss: 0.3794
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9397 - loss: 0.1569 - val_ac

In [11]:
best_baseline_model = keras.models.load_model(baseline_model_path)
best_baseline_model.evaluate(x_test, y_test)
# accuracy: 0.8739 - loss: 0.3364

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8739 - loss: 0.3364


[0.33673349022865295, 0.8725200295448303]

In [18]:
from keras_tuner import HyperParameters

def build_model(hp: HyperParameters) -> keras.Model:
    """build and return a compiled model"""

    inputs = keras.Input(shape=(None,), dtype="int64")

    x = nlp_layers.TokenAndPositionEmbedding(vocabulary_size=vocab_size,
                                             sequence_length=maxlen,
                                             embedding_dim=hp.Choice("embed_dimension", [16, 32, 64, 128])
                                             )(inputs)

    x = nlp_layers.TransformerEncoder(intermediate_dim=hp.Choice("intermediate_dim", [8, 16, 32, 64]),
                                      num_heads=hp.Choice("num_heads", [2, 4, 8, 12])
                                      )(x)

    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model


In [19]:
tuner = keras_tuner.RandomSearch(hypermodel=build_model,
                                 objective="val_accuracy",
                                 max_trials=40,
                                 executions_per_trial=1,
                                 overwrite=True,
                                 directory=trained_model_folder,
                                 project_name="tuning")

tuner.search_space_summary()

Search space summary
Default search space size: 3
embed_dimension (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 128], 'ordered': True}
intermediate_dim (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32, 64], 'ordered': True}
num_heads (Choice)
{'default': 2, 'conditions': [], 'values': [2, 4, 8, 12], 'ordered': True}


In [20]:
tuner.search(x_train, y_train, validation_split=0.2, epochs=2)


Trial 20 Complete [00h 00m 52s]
val_accuracy: 0.852400004863739

Best val_accuracy So Far: 0.8633999824523926
Total elapsed time: 00h 10m 17s


In [16]:
best_hp = tuner.get_best_hyperparameters(1)[0]
best_hp.values

# TODO:
# run 40 trails
# tune bigger value for: intermediate_dim and num_heads

{'embed_dimension': 16, 'intermediate_dim': 64, 'num_heads': 5}

In [ ]:
best_tuned_model = tuner.get_best_models(1)[0]
best_tuned_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 64)            │       1,292,800 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 64)            │          21,088 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,953 (5.01 MB)

 Trainable params: 1,313,953 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
best_tuned_model_path = trained_model_folder / "best_tuned_model.keras"
callbacks = [keras.callbacks.ModelCheckpoint(best_tuned_model_path,
                                             save_best_only=True)]

best_tuned_model.fit(x=x_train, y=y_train,
                     batch_size=64, callbacks=callbacks,
                     validation_split=0.2, epochs=10)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 147s 453ms/step - accuracy: 0.9682 - loss: 0.0937 - val_accuracy: 0.8784 - val_loss: 0.3924
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 200s 448ms/step - accuracy: 0.9865 - loss: 0.0450 - val_accuracy: 0.8708 - val_loss: 0.4978
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 446ms/step - accuracy: 0.9915 - loss: 0.0289 - val_accuracy: 0.8640 - val_loss: 0.5977
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 142s 445ms/step - accuracy: 0.9937 - loss: 0.0205 - val_accuracy: 0.8626 - val_loss: 0.6292
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 443ms/step - accuracy: 0.9968 - loss: 0.0110 - val_accuracy: 0.8652 - val_loss: 0.7226
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 143s 447ms/step - accuracy: 0.9976 - loss: 0.0085 - val_accuracy: 0.8576 - val_loss: 0.8068
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 442ms/step - accuracy: 0.9963 - loss: 0.0106 - val_accuracy: 0.8594 - val_loss: 0.7157
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 145s 464ms/step - accuracy: 0.9967 -

In [ ]:
best_tuned_model = keras.models.load_model(best_tuned_model_path)
best_tuned_model.evaluate(x=x_val, y=y_val)

# accuracy: 0.8775 - loss: 0.2910
# {'embed_dimension': 32, 'intermediate_dim': 16, 'num_heads': 4}  accuracy: 0.8717 - loss: 0.3045

782/782 ━━━━━━━━━━━━━━━━━━━━ 60s 77ms/step - accuracy: 0.8589 - loss: 0.4374


[0.4296519458293915, 0.8609200119972229]

In [ ]:
# TODO:
# save the model on the root folder
# add dense layer after transformer


In [ ]:
#############################################################################

In [ ]:
from keras_tuner import HyperParameters

def build_model(hp: HyperParameters):
    inputs = keras.Input(shape=(None,), dtype="int64")

    # vocabulary_size = hp.Int("vocabulary_size", min_value=20_000, max_value=40_000, step=10_000)
    sequence_length = hp.Int("sequence_length", min_value=300, max_value=600, step=100)
    embed_dimension = hp.Choice("embed_dimension", [32, 64])
    x = nlp_layers.TokenAndPositionEmbedding(
        vocabulary_size=vocab_size,
        sequence_length=sequence_length,
        embedding_dim=embed_dimension
        )(inputs)

    num_heads = hp.Choice("num_heads", [2, 3, 4])
    intermediate_dim = hp.Choice("intermediate_dim", [16, 32, 64])
    x = nlp_layers.TransformerEncoder(
        intermediate_dim=intermediate_dim,
        num_heads=num_heads
        )(x)

    pooling_type = hp.Choice("pooling_type", ["GlobalMaxPolling", "GlobalAveragePooling"])
    with hp.conditional_scope("pooling_type", ["GlobalMaxPolling"]):
        if pooling_type == "GlobalMaxPolling":
            x = layers.GlobalMaxPool1D()(x)
    with hp.conditional_scope("pooling_type", ["GlobalAveragePooling"]):
        if pooling_type == "GlobalAveragePooling":
            x = layers.GlobalAveragePooling1D()(x)

    dropout_rate = hp.Choice("dropout_rate", [0.1, 0.3, 0.5])
    x = layers.Dropout(dropout_rate)(x)

    dense_units = hp.Choice("dense_units", [10, 20, 30])
    x = layers.Dense(dense_units, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)

    optimizer_type = hp.Choice("optimizer_type", ["adam", "rmsprop"])
    model.compile(
        optimizer=optimizer_type,
        loss="binary_crossentropy",
        metrics=["accuracy"])
    return model


build_model(HyperParameters())

<Functional name=functional_1, built=True>

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "/tuned_models/best_tuned_model.keras",
        save_best_only=True)]


best_tuned_model.fit(x=x_train, y=y_train, batch_size=64, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.5747 - loss: 0.6533 - val_accuracy: 0.8752 - val_loss: 0.3050
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9025 - loss: 0.2769 - val_accuracy: 0.8892 - val_loss: 0.2760
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9488 - loss: 0.1656 - val_accuracy: 0.8826 - val_loss: 0.3074
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9704 - loss: 0.1036 - val_accuracy: 0.8672 - val_loss: 0.4804
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9834 - loss: 0.0676 - val_accuracy: 0.8708 - val_loss: 0.5229
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9917 - loss: 0.0398 - val_accuracy: 0.8696 - val_loss: 0.6335
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9937 - loss: 0.0293 - val_accuracy: 0.8538 - val_loss: 0.9273
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9959 - loss: 0.0221 - val_accuracy: 

In [ ]:
final_model = keras.models.load_model("/tuned_models/best_tuned_model.keras")
final_model.evaluate(x=x_val, y=y_val)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8769 - loss: 0.3027


[0.3013143539428711, 0.8768399953842163]

**Download the data**

In [ ]:
keras.utils.get_file(
    origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    cache_dir="./",
    extract=True
)

imdb_dir = Path("./datasets/aclImdb")

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
!tree -d datasets/aclImdb/


datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    ├── pos
    └── unsup

7 directories


remove unsupervised training data, we don't need that here

In [ ]:
!rm -r datasets/aclImdb/train/unsup


In [ ]:
!tree -d datasets/aclImdb/


datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    └── pos

6 directories


quick look at one review

In [ ]:
!cat datasets/aclImdb/train/neg/21_4.txt

What was with all the Turkish actors? No offense but I thought it was all for nothing for all these actors. The film had no script to test any actors acting skill or ability. It demanded next to nothing I bought this film to see Michael Madsen. He is one of my favorite actors but this film was another failure for him. The script was so bad. Their was just nothing to sink your teeth into and all the characters were two dimensional. Madsen tried to act like a hard ass but the script and direction didn't even allow him to do enough with his character to make it more interesting or 3 dimensional.<br /><br />Even the sound effects of the gunfight at the beginning of the film sounded like the noise of paint ball guns when they are fired in a skirmish. It was really weird and they didn't sound like real guns. A video game had better sound effects than this film. There was also a really annoying bloke at the beginning of the film who was a member of the robbery gang. He had this American whini

prepare validation set

In [ ]:
import os, shutil, random

validation_dir = imdb_dir / "validation"
validation_dir.mkdir()
train_dir = imdb_dir / "train"
for category in ("neg", "pos"):
    (validation_dir / category).mkdir()
    files = os.listdir(train_dir / category)
    random.Random(1234).shuffle(files)  # use seed to ensure same dataset through different runs
    num_validation_samples = int(0.2 * len(files))
    validation_files = files[-num_validation_samples:]
    for file in validation_files:
        shutil.move(train_dir / category / file,
                    validation_dir / category / file)


In [ ]:
!tree -d datasets/aclImdb/

datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
├── train
│   ├── neg
│   └── pos
└── validation
    ├── neg
    └── pos

9 directories


In [ ]:
batch_size = 32

# 0 for negative, 1 for positive
train_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/train", batch_size=batch_size)
validation_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/validation", batch_size=batch_size)
test_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_dataset:
    print("inputs.shape: ", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape:  (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"One of Frances Farmer's earliest movies; at 22, she is absolutely beautiful. Bing Crosby is in great voice, but the songs are not his best. Martha Raye and Bob Burns are interesting, but their comedy, probably great in its time, is really corny today. Roy Rogers also appears- in a singing role. In my view only worth watching if you are a Frances Farmer fan, and possibly a Bing Crosby fan.", shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


train a text vectorization layer with unlabeled data

In [ ]:
text_only_train_dataset = train_dataset.map(lambda x, y: x)  # do not need labels to train the text vectorization layer

max_length = 600
max_tokens = 20_000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length)

text_vectorization.adapt(text_only_train_dataset)

prepare integer sequence datasets

In [ ]:
int_train_dataset = train_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_validation_dataset = validation_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_dataset = test_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

use a TransformerEncoder-based model for text classification

train the transformer encoder model

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "trained_models/basic_transformer_encoder.keras",
        save_best_only=True)]

model_1.fit(int_train_dataset,
            validation_data=int_validation_dataset,
            epochs=2,
            callbacks=callbacks)


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.5881 - loss: 0.6518 - val_accuracy: 0.8768 - val_loss: 0.2981
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8908 - loss: 0.2717 - val_accuracy: 0.8906 - val_loss: 0.2923


evaluate the transformer encoder model

In [ ]:
model = keras.models.load_model(
    "trained_models/basic_transformer_encoder.keras")

print(f"Test accuracy: {model.evaluate(int_test_dataset)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8739 - loss: 0.3367
Test accuracy: 0.875


In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
25000 Training sequences
25000 Validation sequences


tune a transfromer encoder

In [ ]:
from keras_tuner import HyperParameters

def build_model(hp: HyperParameters):
    inputs = keras.Input(shape=(None,), dtype="int64")

    # vocabulary_size = hp.Int("vocabulary_size", min_value=20_000, max_value=40_000, step=10_000)
    sequence_length = hp.Int("sequence_length", min_value=300, max_value=600, step=100)
    embed_dimension = hp.Choice("embed_dimension", [32, 64])
    x = nlp_layers.TokenAndPositionEmbedding(
        vocabulary_size=vocab_size,
        sequence_length=sequence_length,
        embedding_dim=embed_dimension
        )(inputs)

    num_heads = hp.Choice("num_heads", [2, 3, 4])
    intermediate_dim = hp.Choice("intermediate_dim", [16, 32, 64])
    x = nlp_layers.TransformerEncoder(
        intermediate_dim=intermediate_dim,
        num_heads=num_heads
        )(x)

    pooling_type = hp.Choice("pooling_type", ["GlobalMaxPolling", "GlobalAveragePooling"])
    with hp.conditional_scope("pooling_type", ["GlobalMaxPolling"]):
        if pooling_type == "GlobalMaxPolling":
            x = layers.GlobalMaxPool1D()(x)
    with hp.conditional_scope("pooling_type", ["GlobalAveragePooling"]):
        if pooling_type == "GlobalAveragePooling":
            x = layers.GlobalAveragePooling1D()(x)

    dropout_rate = hp.Choice("dropout_rate", [0.1, 0.3, 0.5])
    x = layers.Dropout(dropout_rate)(x)

    dense_units = hp.Choice("dense_units", [10, 20, 30])
    x = layers.Dense(dense_units, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)

    optimizer_type = hp.Choice("optimizer_type", ["adam", "rmsprop"])
    model.compile(
        optimizer=optimizer_type,
        loss="binary_crossentropy",
        metrics=["accuracy"])
    return model


build_model(HyperParameters())

<Functional name=functional_3, built=True>

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="tuned_models",
    project_name="tuned_transformer_imdb",
)

tuner.search_space_summary()

Search space summary
Default search space size: 8
sequence_length (Int)
{'default': None, 'conditions': [], 'min_value': 300, 'max_value': 600, 'step': 100, 'sampling': 'linear'}
embed_dimension (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
num_heads (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 4], 'ordered': True}
intermediate_dim (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
pooling_type (Choice)
{'default': 'GlobalMaxPolling', 'conditions': [], 'values': ['GlobalMaxPolling', 'GlobalAveragePooling'], 'ordered': False}
dropout_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.3, 0.5], 'ordered': True}
dense_units (Choice)
{'default': 10, 'conditions': [], 'values': [10, 20, 30], 'ordered': True}
optimizer_type (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop'], 'ordered': False}


In [ ]:
tuner.search(x_train, y_train, validation_split=0.2, epochs=2)

Trial 10 Complete [00h 00m 35s]
val_accuracy: 0.8850000202655792

Best val_accuracy So Far: 0.8863000273704529
Total elapsed time: 00h 08m 56s


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 64)            │       1,305,600 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 64)            │          21,088 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │             650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,327,349 (5.06 MB)

 Trainable params: 1,327,349 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tuner.results_summary()

Results summary
Results in tuned_models/tuned_transformer_imdb
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
sequence_length: 400
embed_dimension: 64
num_heads: 2
intermediate_dim: 32
pooling_type: GlobalAveragePooling
dropout_rate: 0.3
dense_units: 10
optimizer_type: adam
Score: 0.8863000273704529

Trial 09 summary
Hyperparameters:
sequence_length: 400
embed_dimension: 32
num_heads: 2
intermediate_dim: 16
pooling_type: GlobalAveragePooling
dropout_rate: 0.3
dense_units: 30
optimizer_type: adam
Score: 0.8850000202655792

Trial 04 summary
Hyperparameters:
sequence_length: 400
embed_dimension: 32
num_heads: 4
intermediate_dim: 16
pooling_type: GlobalAveragePooling
dropout_rate: 0.3
dense_units: 30
optimizer_type: adam
Score: 0.8833000063896179

Trial 06 summary
Hyperparameters:
sequence_length: 500
embed_dimension: 32
num_heads: 2
intermediate_dim: 16
pooling_type: GlobalAveragePooling
dropout_rate: 0.1
dense_units: 20
optimizer

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)
best_tuned_model = build_model(best_hps[0])
best_tuned_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_1       │ (None, None, 64)            │       1,305,600 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_1                │ (None, None, 64)            │          21,088 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,327,349 (5.06 MB)

 Trainable params: 1,327,349 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# TODO: train more (10 epochs) and add callback to save the best model

best_tuned_model.fit(x=x_train, y=y_train, epochs=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.5699 - loss: 0.6468


In [ ]:
best_tuned_model.evaluate(x=x_val, y=y_val)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8716 - loss: 0.3043


[0.3025885820388794, 0.8724799752235413]

## ***The following is for comaprison***

try closely follow a example on Keras website:  
https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
25000 Training sequences
25000 Validation sequences


In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = nlp_layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim)
x = embedding_layer(inputs)
encoder = nlp_layers.TransformerEncoder(ff_dim, num_heads)
x = encoder(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_3       │ (None, 200, 32)             │         646,400 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_3                │ (None, 200, 32)             │           6,464 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 32)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 20)                  │             660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 2)                   │              42 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 653,566 (2.49 MB)

 Trainable params: 653,566 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#TODO: run the example model with my dataset to see if the size of training set makes the difference

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "trained_models/new_transformer_encoder.keras",
        save_best_only=True)]


history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=2,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)


Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 24ms/step - accuracy: 0.7158 - loss: 0.5174 - val_accuracy: 0.8810 - val_loss: 0.2805
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9245 - loss: 0.1998 - val_accuracy: 0.8712 - val_loss: 0.3056
